In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import tqdm 

In [2]:
import skimage, os
from skimage.morphology import ball, disk, dilation,binary_dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing, binary_opening 
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border, mark_boundaries
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.misc
from glob import glob
from skimage.io import imread


In [3]:
covidx_dir = '/media/ubuntu/MyHDataStor3/datasets/COV19D/'
covidx_img_dir= covidx_dir 
covidx_mask_dir= covidx_dir + 'mask/' 
data_list_dir = '/media/ubuntu/MyHDataStor2/products/COVID-19/ICCV-MAI/3D-CNN-BERT/BERT/datasets/settings/covid/'

print(covidx_dir)
print(covidx_img_dir)
print(covidx_mask_dir)
print(data_list_dir)

/media/ubuntu/MyHDataStor3/datasets/COV19D/
/media/ubuntu/MyHDataStor3/datasets/COV19D/
/media/ubuntu/MyHDataStor3/datasets/COV19D/mask/
/media/ubuntu/MyHDataStor2/products/COVID-19/ICCV-MAI/3D-CNN-BERT/BERT/datasets/settings/covid/


In [4]:
def seedfill(img):

  rows,cols = img.shape 

  img_floodfill = img.copy()
  mask = np.zeros((rows+2,cols+2), np.uint8)

  '''
  # Floodfill from point (0, 0)
  found  = False 
  for x in range(cols): 
    for y in range(rows):
      if img_floodfill[y,x] == 0:
        found = True 
        break  
    if found:
      break 

  #cv2.floodFill(img_floodfill, mask, (x,y), 255);
  ''' 
  

  cv2.floodFill(img_floodfill, mask, (0,0), 255);
  cv2.floodFill(img_floodfill, mask, (0,rows-1), 255);
  cv2.floodFill(img_floodfill, mask, (cols-1,0), 255);
  cv2.floodFill(img_floodfill, mask, (cols-1,rows-1), 255);

  return img_floodfill 


In [5]:
def body_contour(binary_image):
    """Helper function to get body contour"""
    contours, hier = cv2.findContours(binary_image, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    body_idx = np.argmax(areas)
    return contours[body_idx]


def auto_body_crop(image, plot=False):
    """Roughly crop an image to the body region"""
    # Create initial binary image
    filt_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    if plot == True:
        f, plots = plt.subplots(3, 1, figsize=(5, 40))        
        plots[0].axis('off')
        plots[0].imshow(filt_image, cmap=plt.cm.bone)     
    
    
    thresh = cv2.threshold(filt_image[filt_image > 0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    #print(thresh) 
    
    bin_image = np.uint8(filt_image > thresh)  
    #bin_image2 = np.uint8(filt_image > 240)         
    
    
    if plot == True:
        plots[1].axis('off')
        plots[1].imshow(bin_image, cmap=plt.cm.bone)     
    
    
    erode_kernel = np.ones((7, 7), dtype=np.uint8)
    bin_image = cv2.erode(bin_image, erode_kernel)
    
    #print(np.unique(bin_image))
    if plot == True:
        plots[2].axis('off')
        plots[2].imshow(bin_image, cmap=plt.cm.bone)     
        
    # Find body contour
    body_cont = body_contour(bin_image).squeeze()
    
    '''
    #find the lung rectangle        
    binary2 = ((1-bin_image)*255).astype(np.uint8) 
    kernel = np.ones((5,5), np.uint8) 
    binary2 = cv2.dilate(binary2, kernel, iterations=2) 
    binary2 = cv2.erode(binary2, kernel, iterations=2) 
    binary2 = seedfill(binary2) 
    binary2 = cv2.bitwise_not(binary2)    
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(binary2, cmap=plt.cm.bone) 
    '''    
    
    #print(body_cont)

    # Get bbox
    xmin = body_cont[:, 0].min()
    xmax = body_cont[:, 0].max() + 1
    ymin = body_cont[:, 1].min()
    ymax = body_cont[:, 1].max() + 1
       
    return bin_image, (xmin, ymin, xmax, ymax)


def auto_segment(image, plot=False):
    """Roughly crop an image to the body region"""
    # Create initial binary image
    filt_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    if plot == True:
        f, plots = plt.subplots(9, 1, figsize=(5, 40))        
        plots[0].axis('off')
        plots[0].imshow(filt_image, cmap=plt.cm.bone)     
    
    
    thresh = cv2.threshold(filt_image[filt_image > 0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    print(thresh) 
    
    bin_image = np.uint8(filt_image > thresh)  
    #bin_image2 = np.uint8(filt_image > 240)             
    
    if plot == True:
        plots[1].axis('off')
        plots[1].imshow(bin_image, cmap=plt.cm.bone)     
        
    erode_kernel = np.ones((7, 7), dtype=np.uint8)
    bin_image = cv2.erode(bin_image, erode_kernel)
    #print(np.unique(bin_image))
    if plot == True:
        plots[2].axis('off')
        plots[2].imshow(bin_image, cmap=plt.cm.bone)     
        
 
        
    # Find body contour
    body_cont = body_contour(bin_image).squeeze()
    
    #print(body_cont)

    # Get bbox
    xmin = body_cont[:, 0].min()
    xmax = body_cont[:, 0].max() + 1
    ymin = body_cont[:, 1].min()
    ymax = body_cont[:, 1].max() + 1
       
        
    bin_image_inv = (1 - bin_image)*255 
    bin_image_inv = seedfill(bin_image_inv)/255 
    cleared = clear_border(bin_image_inv)
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(cleared, cmap=plt.cm.bone)     
    
    label_image = label(cleared)
    if plot == True:
        plots[4].axis('off')
        plots[4].imshow(label_image, cmap=plt.cm.bone) 
        
    '''
    Step 4: Keep the labels with 2 largest areas.
    '''
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       label_image[coordinates[0], coordinates[1]] = 0

    binary = label_image > 0
    if plot == True:
        plots[5].axis('off')
        plots[5].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 5: Erosion operation with a disk of radius 2. This operation is 
    seperate the lung nodules attached to the blood vessels.
    '''
    selem = disk(7)
    binary = binary_erosion(binary, selem)
    if plot == True:
        plots[6].axis('off')
        plots[6].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 6: Closure operation with a disk of radius 10. This operation is 
    to keep nodules attached to the lung wall.
    '''
    selem = disk(10)
    binary = binary_closing(binary, selem)
    if plot == True:
        plots[7].axis('off')
        plots[7].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 7: Fill in the small holes inside the binary mask of lungs.
    '''

    binary1 = binary.copy() 

    edges = roberts(binary)
    binary = ndi.binary_fill_holes(edges)
    if plot == True:
        plots[8].axis('off')
        plots[8].imshow(binary, cmap=plt.cm.bone) 
    
    
    return binary, binary1, (xmin, ymin, xmax, ymax)


In [6]:
def refine_mask(m_full):
    
    mask2 = cv2.imread(m_full, cv2.IMREAD_UNCHANGED)
    mask2 = (mask2/255).astype(np.uint8) 
    
    selem = disk(10)
    mask2_2  = binary_closing(mask2, selem)                
    label_image = label(mask2_2)
    
    '''
    Step 4: Keep the labels with 2 largest areas.
    '''
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                    label_image[coordinates[0], coordinates[1]] = 0
    mask2_3 = label_image > 0               
    
    edges = roberts(mask2_3) 
    mask2_4 = ndi.binary_fill_holes(edges)
                
    return mask2,mask2_4

In [7]:
#subsets = ['val','train']  #for test dataset, UNet segmentation and refinement are combined. 
subsets = ['test']

from tqdm import tqdm

debug = False
split = 1 

for subset in subsets:     

    list_file = data_list_dir + '{}_rgb_split{}.txt'.format(subset,split) 
    lines = open(list_file,"r").readlines() 

    pbar = tqdm(total = len(lines))
    
    for l in lines:
        
        print(l)
        
        pbar.update()
        
        l_splits = l.split()  
                
        l_path = l_splits[0]        
        c = l_splits[1]
        first_slice = int(l_splits[2])
        last_slice = int(l_splits[3])

        count = 0 
        for f in range(first_slice,last_slice+1):
            
            count +=1 
            
            #if count>10:
            #    break 
            
            fn = "{}/{}.jpg".format(l_path,f)
            f_full = covidx_img_dir + fn
            m_full = covidx_img_dir + 'mask/' + fn
            m_full_new = m_full+'_refined.jpg'            
            #print(f_full,m_full,m_full_new)
            
            if not os.path.exists(m_full_new) :
                mask0,mask = refine_mask(m_full)                
                cv2.imwrite(m_full_new, (mask*255).astype(np.uint8) )

                if False: 
                    fig, (ax1,ax2) = plt.subplots(1,2)
                    ax1.imshow(mask0*255,cmap='gray')
                    ax2.imshow(mask*255,cmap='gray')                
                


  0%|          | 0/5546 [00:00<?, ?it/s]

test/part1_1_set_test_ECCV/ct_scan_3362 covid 16 47 0 55 512 408



  0%|          | 2/5546 [00:02<2:07:28,  1.38s/it]

test/part1_0_test_set_ECCV/ct_scan_474 covid 31 78 5 155 488 441



  0%|          | 3/5546 [00:06<3:23:59,  2.21s/it]

...

test/part1_1_set_test_ECCV/ct_scan_3208 covid 79 261 35 147 459 433



100%|█████████▉| 5545/5546 [12:06:43<00:09,  9.43s/it]

test/part1_0_1_test_set_ECCV/ct_scan_950 covid 57 160 13 112 506 412



100%|██████████| 5546/5546 [12:06:52<00:00,  9.26s/it]

test/part1_0_2_test_set_ECCV/ct_scan_2192 covid 52 200 0 121 512 505

